## Setting the Database

### Loading DataFrame & Change to desired input format for DB

In [2]:
## loading the data 
import pandas as pd
## defining the path 
PATH = "./data/clean_data.csv"
## loading 
df = pd.read_csv(PATH)
## reseting indeces
dfa = df.reset_index(drop=True)

## number of materials 
mats = [x for x in df.columns if "M-" in x]
n_mats = len(mats)
print(n_mats)
## current dataframe: Processes | Material 1| Material 2| Material 3| ... 
## convert to Processes | Materials | Values
long_df = pd.wide_to_long(dfa, ["M-"], i="Processes", j="Material").reset_index().rename(columns={"M-": "Consumption"})
## add Input or Output column
## x > 0 -> Input, x < 0 -> Output, x = 0 -> None
long_df['IO'] = long_df['Consumption'].apply(lambda x: 1 if x > 0 else -1 if x < 0 else 0)
## adding the text to it in case
long_df['IO_txt'] = long_df['IO'].apply(lambda x: "Input" if x == 1 else "Output" if x == -1 else "None")


long_df['Processes'] = list(map(lambda x: f'P{int(x.split("-")[1])}',long_df['Processes']))
long_df['Material'] = list(map(lambda x: f"M{x}",long_df['Material']))


## the dataframe is per process, 
long_df = long_df.sort_values(by=['Processes','Material'])
long_df_nz = long_df.query("Consumption != 0")


## Used Materials
input_materials = long_df_nz.query("IO == 1")
used_materials_per_process = input_materials.groupby("Processes")['Material'].apply(list).reset_index()
used_materials_per_process.rename(columns={"Material": "Inputs"}, inplace=True)
## Energy to Input 
used_consumption_materials_per_process = input_materials.groupby("Processes")['Consumption'].apply(list).reset_index()
used_consumption_materials_per_process.rename(columns={"Consumption": "InputCost"}, inplace=True)
## merge 
used_materials_df = input_materials.merge(used_materials_per_process, on="Processes", how="left")
used_materials_df = used_materials_df.merge(used_consumption_materials_per_process, on="Processes", how="left")


## Produced materials
output_materials = long_df_nz.query("IO == -1")
produced_materials_per_process = output_materials.groupby("Processes")['Material'].apply(list).reset_index()
produced_materials_per_process.rename(columns={"Material": "Outputs"}, inplace=True)
## Consumption Needed
produced_consumption_materials_per_process = output_materials.groupby("Processes")['Consumption'].apply(list).reset_index()
produced_consumption_materials_per_process.rename(columns={"Consumption": "OutputCost"}, inplace=True)
## merge 
produced_materials_df = output_materials.merge(produced_materials_per_process, on="Processes", how="left")
produced_materials_df = produced_materials_df.merge(produced_consumption_materials_per_process, on="Processes", how="left")
produced_materials_df


## tmp1 : used_materials drop Material	Consumption	IO	IO_txt
tmp1 = used_materials_df.drop(columns=["Material", "Consumption", "IO", "IO_txt"])
## tmp2: produced_materials_df
tmp2 = produced_materials_df.drop(columns=["Material", "Consumption", "IO", "IO_txt"])
tmp3 = tmp1.merge(tmp2, on="Processes", how="left")

# output_cost = lambda x: x if x>0 else 0
# input_cost = lambda x: x if x<0 else 0
# ## map the function to the list
tmp3['TotalOutputCost'] = tmp3['OutputCost'].apply(lambda x: sum(x))
tmp3['TotalInputCost'] = tmp3['InputCost'].apply(lambda x: sum(x))
## drop the duplicates 
tmp3 = tmp3.drop_duplicates('Processes')
tmp3.head()

to_use = long_df_nz.drop(columns=['IO','IO_txt'])
to_use.to_csv("./data/final_df_for_db.csv", index=False)

20


### Start the Database

Essentially doing this 

1) Database 
2) Connection to DB with action 
3) Ask queries to the system

In [3]:
## autoreload 
%load_ext autoreload
%autoreload 2


In [11]:
## importing the connectors 
from db.connector import DBConnector
## load the environment variables
from db.db_helpers import load_env_vars, clear_db

## import the materials 
from db.Materials import (Material, material_payload,
                          populate_materials_from_df,
                          add_material,remove_material,
                          update_material_name,update_material_quantity,
                          add_material_relationship)

## import the processes 
from db.Processes import (Process, process_payload,
                          populate_process_from_df,add_process)

In [5]:
## loading environment variables to connect to the database 
env_vars = load_env_vars()
## instantiate the connector 
connector = DBConnector(**env_vars)
## start the connection 
connector._connect()
## close the connection 
#connector._close()
## getting all the Material nodes
#Material.nodes.all()
clear_db()

Loaded environment variables
Connected to the database
Database cleared


([], [])

### Testing the db 

In [6]:
hg_dict = {"uuid": "M99",
                 "name": "Hg",
                 "quantity": 100, "unit": "ton",
                 "cost": 1000000000,
                 "description":"Material 99 - is pure Silver"} ## the dictionary for the first material

hg,status = add_material(hg_dict,verbose=True)


ur_dict = {"uuid": "M98",
                 "name": "Ur",
                 "quantity": 999, "unit": "ton",
                 "cost": 99999999999,
                 "description":"Material 98 - is pure Uranium, used in Max's Laboratory"} ## the dictionary for the first material

ur,status2 = add_material(ur_dict,verbose=True)

print(Material.nodes.all(),end='\n')
#clear_db()

Material: M99-Hg added
Material: M98-Ur added
[<Material: {'uuid': 'M99', 'name': 'Hg', 'quantity': 100, 'unit': 'ton', 'cost': 1000000000, 'description': 'Material 99 - is pure Silver', 'id': 0}>, <Material: {'uuid': 'M98', 'name': 'Ur', 'quantity': 999, 'unit': 'ton', 'cost': 99999999999, 'description': "Material 98 - is pure Uranium, used in Max's Laboratory", 'id': 1}>]


In [7]:
clear_db()
## Silver
hg,status = add_material(hg_dict,verbose=False)
## Uranium 
ur,status2 = add_material(ur_dict,verbose=False)
## create a relationship 
rel_status1 = add_material_relationship(hg, ur) ## hg --> ur 
rel_status2 = add_material_relationship(ur, hg) ## ur --> hg 

## update the material name 
old_uuid = "M99"
new_name = 'K'
upname1 = update_material_name(material_id=old_uuid, new_name=new_name)

Database cleared
Added relationship between M99 and M98
Added relationship between M98 and M99
Updated Material: M99 with new name: K


### Populating the db

In [8]:
populate_materials_from_df(df=to_use, verbose=True)

Material: M1-M1__ added
Material: M12-M12__ added
Material: M15-M15__ added
Material: M18-M18__ added
Material: M19-M19__ added
Material: M3-M3__ added
Material: M5-M5__ added
Material: M10-M10__ added
Material: M12 already exists
Material: M15 already exists
Material: M18 already exists
Material: M2-M2__ added
Material: M20-M20__ added
Material: M3 already exists
Material: M6-M6__ added
Material: M8-M8__ added
Material: M1 already exists
Material: M10 already exists
Material: M11-M11__ added
Material: M15 already exists
Material: M16-M16__ added
Material: M18 already exists
Material: M19 already exists
Material: M20 already exists
Material: M3 already exists
Material: M4-M4__ added
Material: M6 already exists
Material: M8 already exists
Material: M1 already exists
Material: M11 already exists
Material: M15 already exists
Material: M16 already exists
Material: M17-M17__ added
Material: M18 already exists
Material: M3 already exists
Material: M4 already exists
Material: M6 already exist

In [9]:
## populate the processes
populate_process_from_df(df=to_use, verbose=True)

Process P1 added
Process P1 already exists
Process P1 already exists
Process P1 already exists
Process P1 already exists
Process P1 already exists
Process P1 already exists
Process P10 added
Process P10 already exists
Process P10 already exists
Process P10 already exists
Process P10 already exists
Process P10 already exists
Process P10 already exists
Process P10 already exists
Process P10 already exists
Process P11 added
Process P11 already exists
Process P11 already exists
Process P11 already exists
Process P11 already exists
Process P11 already exists
Process P11 already exists
Process P11 already exists
Process P11 already exists
Process P11 already exists
Process P11 already exists
Process P11 already exists
Process P12 added
Process P12 already exists
Process P12 already exists
Process P12 already exists
Process P12 already exists
Process P12 already exists
Process P12 already exists
Process P12 already exists
Process P12 already exists
Process P12 already exists
Process P13 added

In [10]:

Material.nodes.all()

[<Material: {'uuid': 'M1', 'name': 'M1__', 'quantity': -73, 'unit': None, 'cost': None, 'description': None, 'id': 0}>,
 <Material: {'uuid': 'M12', 'name': 'M12__', 'quantity': -60, 'unit': None, 'cost': None, 'description': None, 'id': 1}>,
 <Material: {'uuid': 'M99', 'name': 'K', 'quantity': 100, 'unit': 'ton', 'cost': 1000000000, 'description': 'Material 99 - is pure Silver', 'id': 2}>,
 <Material: {'uuid': 'M98', 'name': 'Ur', 'quantity': 999, 'unit': 'ton', 'cost': 99999999999, 'description': "Material 98 - is pure Uranium, used in Max's Laboratory", 'id': 3}>,
 <Material: {'uuid': 'M15', 'name': 'M15__', 'quantity': -26, 'unit': None, 'cost': None, 'description': None, 'id': 4}>,
 <Material: {'uuid': 'M18', 'name': 'M18__', 'quantity': -9, 'unit': None, 'cost': None, 'description': None, 'id': 5}>,
 <Material: {'uuid': 'M19', 'name': 'M19__', 'quantity': -15, 'unit': None, 'cost': None, 'description': None, 'id': 6}>,
 <Material: {'uuid': 'M3', 'name': 'M3__', 'quantity': 59, 'un

In [12]:

Process.nodes.all()

[<Process: {'uuid': 'P1', 'name': 'P1__XX', 'description': None, 'total_input_cost': None, 'total_output_cost': None, 'id': 22}>,
 <Process: {'uuid': 'P10', 'name': 'P10__XX', 'description': None, 'total_input_cost': None, 'total_output_cost': None, 'id': 23}>,
 <Process: {'uuid': 'P11', 'name': 'P11__XX', 'description': None, 'total_input_cost': None, 'total_output_cost': None, 'id': 24}>,
 <Process: {'uuid': 'P12', 'name': 'P12__XX', 'description': None, 'total_input_cost': None, 'total_output_cost': None, 'id': 25}>,
 <Process: {'uuid': 'P13', 'name': 'P13__XX', 'description': None, 'total_input_cost': None, 'total_output_cost': None, 'id': 26}>,
 <Process: {'uuid': 'P14', 'name': 'P14__XX', 'description': None, 'total_input_cost': None, 'total_output_cost': None, 'id': 27}>,
 <Process: {'uuid': 'P15', 'name': 'P15__XX', 'description': None, 'total_input_cost': None, 'total_output_cost': None, 'id': 28}>,
 <Process: {'uuid': 'P16', 'name': 'P16__XX', 'description': None, 'total_inpu